In [1]:
import itertools
import numpy as np

import hoomd
import gsd.hoomd
import coxeter

#### vertices template

In [2]:
h=0.5
verts = [(np.cos(0), np.sin(0), h),
         (np.cos(np.pi/3), np.sin(np.pi/3), h),
         (np.cos(2*np.pi/3), np.sin(2*np.pi/3), h),
         (np.cos(3*np.pi/3), np.sin(3*np.pi/3), h),
         (np.cos(4*np.pi/3), np.sin(4*np.pi/3), h),
         (np.cos(5*np.pi/3), np.sin(5*np.pi/3), h),
         (np.cos(0), np.sin(0), -h),
         (np.cos(np.pi/3), np.sin(np.pi/3), -h),
         (np.cos(2*np.pi/3), np.sin(2*np.pi/3), -h),
         (np.cos(3*np.pi/3), np.sin(3*np.pi/3), -h),
         (np.cos(4*np.pi/3), np.sin(4*np.pi/3), -h),
         (np.cos(5*np.pi/3), np.sin(5*np.pi/3), -h)]

#### rescaling particle by unit volume

In [3]:
particle = coxeter.shapes.ConvexPolyhedron(verts)
scale = (particle.volume)**(1/3)
verts /= scale

new_particle = coxeter.shapes.ConvexPolyhedron(verts)
new_particle.volume

1.0

#### placing at simple cubic lattice points

In [4]:
m=4
spacing = 2.2 # > particle size
L = m * spacing

N_tot = m**3
N_particles = N_tot

x = np.linspace(-L/2, L/2, m, endpoint=False) + spacing/2
position = list(itertools.product(x, repeat=3))[:N_particles]

#### snapshot

In [5]:
snapshot = gsd.hoomd.Frame()
snapshot.particles.types = ['A']
snapshot.particles.N = N_particles
snapshot.particles.position = position
snapshot.particles.orientation = [1, 0, 0, 0]*N_particles

moment_inertia = particle.inertia_tensor.diagonal().tolist()
snapshot.particles.moment_inertia = moment_inertia * N_particles

snapshot.configuration.box = [L, L, L, 0, 0, 0]
snapshot.particles.type_shapes = [{'type': 'ConvexPolyhedron',
                                   'rounding_radius': 0.,
                                   'vertices': verts.tolist()}]

#### gsd

In [6]:
with gsd.hoomd.open(name='./DATA/lattice.gsd', mode='wb') as f:
    f.append(snapshot)